## Notebook for data clean up
* Remove bad thermocouple data
* Remove down periods
* Remove invalid entries like NaN

In [1]:
 # Import required libraries for data manipulation and visualization
import pandas as pd
import numpy as np
import plotly.graph_objects as go

In [3]:
# read process data from hdf5 file
df = pd.read_hdf('../data/process-data.h5')

# examine the columns and first five rows
df.head()

,timestamp,air.flow,air.temp,air.frac,fuel.flow,tc1,tc2,tc3,tc4,tcf,...,f.co2,f.o2,f.ch4,f.nox,f.co,spec,stoppage,hub,shift,trial
0,2020-01-01 00:00:00,2479,28.65,0.24,390,29,1450,1866,1832,1758,...,0.031799,2.784636e-13,0.003849,0.000788,0.000289,CIA,False,22,B,1.0
1,2020-01-01 00:01:00,2479,28.65,0.24,390,26,1450,1863,1835,1760,...,0.032406,2.678304e-13,0.003662,0.000790,0.000314,CIA,False,22,B,1.0
2,2020-01-01 00:02:00,2479,28.65,0.24,390,26,1446,1867,1833,1760,...,0.031569,2.608832e-13,0.003831,0.000793,0.000308,CIA,False,22,B,1.0
3,2020-01-01 00:03:00,2479,28.65,0.24,390,27,1448,1862,1837,1760,...,0.032599,2.557492e-13,0.003604,0.000792,0.000306,CIA,False,22,B,1.0
4,2020-01-01 00:04:00,2479,28.65,0.24,390,26,1448,1867,1836,1755,...,0.031396,2.694801e-13,0.003752,0.000799,0.000300,CIA,False,22,B,1.0


In [4]:
# examine statistics of thermocouple data
df[['tc1','tc2','tc3','tc4','tc4']].describe()

,tc1,tc2,tc3,tc4,tc4
count,133921.000000,133921.000000,133921.000000,133921.000000,133921.000000
mean,99.168950,443.279934,1554.673210,1604.034289,1604.034289
std,559.469778,695.425814,863.410878,542.676655,542.676655
min,22.000000,27.000000,25.000000,28.000000,28.000000
25%,30.000000,131.000000,1085.000000,1218.000000,1218.000000
50%,57.000000,171.000000,1328.000000,1482.000000,1482.000000
75%,99.000000,345.000000,1866.000000,1912.000000,1912.000000
max,9999.000000,9999.000000,9999.000000,3039.000000,3039.000000


In [11]:
# get the shape of the raw dataframe
print('Before filtering:', df.shape)

# create a new dataframe using only rows where all thermocouples are valid
# the query removes data values greater than 5000 and also drops rows where the stoppage value is True
fildf = df.query('tc1 < 5000 and tc2 < 5000 and tc3 < 5000 \
    and tc4 < 5000 and tcf < 5000 \
    and stoppage == False').reset_index(drop=True)

print('After filtering:', fildf.shape)

# drop nan or empty values
print(fildf.isna().sum())
fildf = fildf.dropna()

# get the new dataframe shape
print('After filtering:', fildf.shape)

Before filtering: (133921, 21)
After filtering: (131699, 21)
timestamp    0
air.flow     0
air.temp     0
air.frac     0
fuel.flow    0
tc1          0
tc2          0
tc3          0
tc4          0
tcf          0
f.h2o        0
f.co2        0
f.o2         0
f.ch4        0
f.nox        0
f.co         0
spec         0
stoppage     0
hub          0
shift        0
trial        0
dtype: int64
After filtering: (131699, 21)


In [12]:
# Complete filtering and data cleanup tasks
# write filtered dataframe to hdf5 file
fildf.to_hdf('../data/clean-data.h5',key='df',mode='w')

/var/folders/r5/ph_v4dmx447b58v1n2xjl46h0000gn/T/ipykernel_18553/3044948918.py:2: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block3_values] [items->Index(['spec', 'stoppage', 'shift'], dtype='object')]

  fildf.to_hdf('../data/clean-data.h5',key='df',mode='w')


In [13]:
# For Self Use: Need to see the dataset in csv
df = pd.read_hdf('../data/clean-data.h5')
df.to_csv('../data/clean-data.csv', index=False)